In [ ]:
import arviz as az
task = "total-task-hierar_binom_n_var_inference_data.json"
data = az.from_json(f"outputs/v3_prolific/cognitive_battery/loadblindness/{task}")

In [ ]:
import matplotlib.pyplot as plt
axs = az.plot_trace(data, var_names='delta_zpdes')

In [ ]:
# Plot trace using arviz
fig, axs = plt.subplots(figsize=(12, 3))  # Adjust the figsize here (width, height) in inches

az.plot_posterior(data, var_names='delta_zpdes', color='black', ax=axs, hdi_prob='hide', point_estimate=None)

# Generate synthetic data
data_normal = np.random.normal(loc=0, scale=0.05, size=1000)

# Calculate parameters for the normal distribution
mu = np.mean(data_normal)
sigma = np.std(data_normal)
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
normal_distribution = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp(-(x - mu)**2 / (2 * sigma**2))

# Add the normal distribution curve to the first subplot
# Note: axs structure needs to be verified, assuming here it's a 2D array where each variable and chain gets its own row and column.
axs.plot(x, normal_distribution, color='grey', linestyle='--')
axs.spines['top'].set_visible(True)
axs.spines['right'].set_visible(True)
axs.spines['left'].set_visible(True)
plt.title("")
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
df = pd.read_csv('merged_F1_intra.csv')
df['trajectory'] = df.apply(lambda row: row['trajectory'].strip('[]').split(), axis=1)# Expand the 'trajectory' column into 48 separate columns
trajectory_expanded = pd.DataFrame(df['trajectory'].tolist(), columns=[f'trial_{i+1}' for i in range(48)])

# Concatenate the new columns with the original DataFrame
df = pd.concat([df.drop(columns=['trajectory']), trajectory_expanded], axis=1)
fixed_columns = ['participant_id', 'condition', 'study']

# Initialize list to collect the transformed rows
transformed_rows = []

for _, row in df.iterrows():
    for ii, i in enumerate(range(0, 48, 12)):
        new_row = row[fixed_columns].to_dict()
        trial_columns = {str(j+1): row[f'trial_{i+j+1}'] for j in range(12)}
        new_row.update(trial_columns)
        new_row.update({'session_id': ii})
        transformed_rows.append(new_row)

# Create a new DataFrame from the transformed rows
df = pd.DataFrame(transformed_rows)
df.to_csv('long_format.csv')

In [ ]:
# Compute the correlation matrix
corr_matrix = df[columns_score].corr()

# Create a heatmap plot
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

In [ ]:
from sklearn.decomposition import PCA, NMF
import numpy as np
# Combine all sessions for PCA
levels_data = df[[str(i) for i in range(1, 13)]].astype(float)

# Perform PCA on the combined dataset
pca = PCA(n_components=10)
nmf = NMF(n_components=10, init='random', random_state=42)

principal_components = pca.fit_transform(levels_data)
explained_variance_ratio = pca.explained_variance_ratio_[0]
df['PCA1'] = principal_components
print("PCA", pca.explained_variance_ratio_)

nmf_components = nmf.fit_transform(levels_data)
W = nmf.fit_transform(levels_data)
H = nmf.components_

# Project each session using the found components
df['NMF1'] = W

# Calculate the Frobenius norm of the original data and the approximation
frobenius_norm_original = np.linalg.norm(levels_data)
frobenius_norm_approximation = np.linalg.norm(np.dot(W, H))

# Calculate explained variance ratio (approximation)
explained_variance_ratio = 1 - (frobenius_norm_approximation / frobenius_norm_original)
print(explained_variance_ratio)

df

In [ ]:
df_pivot = df.pivot(index=['participant_id', 'study', 'condition'], columns='session_id', values='PCA1')
df_pivot.to_csv("PCA_F1_intra_all.csv")

In [ ]:
import semopy as sem
model_simple = """
        # Latent variables
        Intercept =~ 1*s_0 + 1*s_1 + 1*s_2 + 1*s_3
        Slope =~ 0*s_0 + 1*s_1 + 2*s_2 + 3*s_3
    
        # Allow intercept and slope to correlate
        Intercept ~~ Slope
    
        # Regression paths
        s_0 ~ Intercept
        s_1 ~ Intercept + Slope
        s_2 ~ Intercept + Slope
        s_3 ~ Intercept + Slope
"""

In [49]:
import pandas as pd
intra = pd.read_csv('aggregated_wide_format_with_averages.csv')
for i in range(4):
    intra[f"intra_session{i}"] = intra[[col for col in intra.columns if f"session_id_{i}_" in col]].mean(axis=1)
intra

,participant_id,condition,study,Unnamed: 0,session_id_0_1,session_id_1_1,session_id_2_1,session_id_3_1,session_id_0_2,session_id_1_2,...,average_session_id_2_easy,average_session_id_2_medium,average_session_id_2_hard,average_session_id_3_easy,average_session_id_3_medium,average_session_id_3_hard,intra_session0,intra_session1,intra_session2,intra_session3
0,513,baseline,v3_utl,289.5,4.0,4.0,4.000000,4.0,3.0,2.5,...,3.500000,3.034722,3.000000,3.625000,3.125000,2.291667,2.630141,2.738782,3.178241,3.013889
1,516,baseline,v3_utl,293.5,4.0,4.0,3.666667,4.0,3.5,4.0,...,3.541667,3.312500,2.416667,3.625000,2.625000,2.875000,2.998106,3.114899,3.090278,3.041667
2,517,baseline,v3_utl,297.5,4.0,4.0,4.000000,4.0,4.0,4.0,...,4.000000,3.812500,2.833333,3.916667,3.839286,3.066667,3.565025,3.506944,3.548611,3.607540
3,518,zpdes,v3_utl,301.5,3.0,4.0,3.500000,4.0,3.5,2.0,...,3.000000,2.437500,2.370455,3.375000,2.437500,2.590909,2.328662,2.740657,2.602652,2.801136
4,520,baseline,v3_utl,305.5,4.0,4.0,4.000000,4.0,4.0,4.0,...,3.875000,3.500000,2.862587,3.625000,3.187500,2.905303,3.180934,3.626887,3.412529,3.239268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,674,zpdes,v3_prolific,273.5,3.0,4.0,4.000000,4.0,4.0,3.0,...,3.750000,3.291667,2.916667,3.750000,3.750000,2.526515,2.983974,3.439394,3.319444,3.342172
118,676,baseline,v3_prolific,277.5,4.0,4.0,4.000000,4.0,4.0,3.5,...,3.875000,3.937500,2.849359,3.750000,3.437500,3.018357,3.333814,3.520785,3.553953,3.401952
119,678,zpdes,v3_prolific,281.5,3.8,4.0,4.000000,4.0,4.0,4.0,...,3.750000,3.441468,2.634149,4.000000,3.027778,2.706993,3.337115,3.386623,3.275206,3.244924
120,679,baseline,v3_prolific,285.5,4.0,4.0,4.000000,3.0,3.0,4.0,...,3.875000,3.187500,2.654196,3.500000,3.479167,3.138195,3.361742,3.236364,3.238899,3.372454


In [50]:
# Short script to retrieve the cog battery score to add it to df
import json
import numpy as np 

from analysis_scripts.cognitive_battery.PCA.fit_models import format_conditions
from analysis_scripts.cognitive_battery.PCA.normalize import normalize_data
from analysis_scripts.cognitive_battery.PCA.utils import get_df

cog_battery = get_df(['v3_prolific', 'v3_utl'])

with open('analysis_scripts/cognitive_battery/PCA/config/conditions.JSON', 'r') as f:
    conditions = json.load(f)
tasks, tasks_nb, kept_columns = format_conditions(conditions)

rt_cols = [col for col in intra.columns if "rt" in col and "participant" not in col]
cog_battery[rt_cols] = cog_battery[rt_cols].replace(0, np.nan).apply(lambda x: 1 / x)
cog_battery[tasks] = cog_battery[tasks].fillna(cog_battery[tasks].mean())
cog_battery[tasks] = normalize_data(cog_battery, tasks, tasks, shuffle=False)
cog_battery['score'] = cog_battery[tasks].mean(axis=1)
cog_battery = cog_battery[kept_columns+['score']]
cog_battery = cog_battery.pivot(index=['participant_id', 'condition', 'study'], columns='task_status', values='score').reset_index()
cog_battery

/var/folders/m9/p4hnlvpx2vb5yr2f8kxl9sl40000gp/T/ipykernel_69459/3038619520.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cog_battery['score'] = cog_battery[tasks].mean(axis=1)


task_status,participant_id,condition,study,POST_TEST,PRE_TEST
0,513,baseline,v3_utl,0.142521,0.033361
1,516,baseline,v3_utl,-0.152519,-0.442244
2,517,baseline,v3_utl,-0.129314,-0.220924
3,518,zpdes,v3_utl,-0.619448,-0.925295
4,520,baseline,v3_utl,-0.002864,-0.553478
...,...,...,...,...,...
118,672,baseline,v3_prolific,0.101828,-0.209551
119,674,zpdes,v3_prolific,0.336924,0.243047
120,676,baseline,v3_prolific,0.419780,-0.110188
121,677,baseline,v3_prolific,0.802916,0.742002


In [51]:
nasa_prolif = pd.read_csv("data/v3_prolific/questionnaires/nasa_tlx/nasa_tlx.csv")
nasa_prolif['study']='v3_prolific'
nasa_utl = pd.read_csv("data/v3_utl/questionnaires/nasa_tlx_v3_utl/nasa_tlx_all.csv")
nasa_utl['study'] = 'v3_utl'
nasa = pd.concat([nasa_prolif, nasa_utl])
nasa = nasa[['id_participant', 'session_id', 'condition', 'study', 'load_index']]
nasa = nasa.pivot(index=['id_participant', 'condition', 'study'], columns='session_id', values='load_index').reset_index()
nasa = nasa.rename(columns={i:f"load_index_s{i}" for i in range(1, 9)})
nasa = nasa.rename(columns={"id_participant": "participant_id"})
nasa['load_index'] = nasa[[f"load_index_s{i}" for i in range(1, 9)]].mean(axis=1)
nasa

session_id,participant_id,condition,study,load_index_s1,load_index_s2,load_index_s3,load_index_s4,load_index_s5,load_index_s6,load_index_s7,load_index_s8,load_index
0,513,baseline,v3_utl,60.0,65.0,82.0,78.0,77.0,78.0,87.0,75.0,75.250
1,516,baseline,v3_utl,41.0,66.0,57.0,36.0,38.0,39.0,28.0,32.0,42.125
2,517,baseline,v3_utl,67.0,56.0,70.0,51.0,57.0,56.0,51.0,51.0,57.375
3,518,zpdes,v3_utl,74.0,68.0,70.0,52.0,48.0,29.0,70.0,46.0,57.125
4,520,baseline,v3_utl,49.0,59.0,67.0,65.0,71.0,74.0,67.0,77.0,66.125
...,...,...,...,...,...,...,...,...,...,...,...,...
116,671,baseline,v3_prolific,52.0,68.0,52.0,55.0,51.0,51.0,51.0,55.0,54.375
117,672,baseline,v3_prolific,81.0,64.0,69.0,80.0,66.0,72.0,71.0,58.0,70.125
118,674,zpdes,v3_prolific,56.0,54.0,58.0,50.0,49.0,53.0,44.0,46.0,51.250
119,677,baseline,v3_prolific,68.0,25.0,43.0,68.0,63.0,56.0,58.0,46.0,53.375


In [52]:
ues_prolif = pd.read_csv("data/v3_prolific/questionnaires/ues/ues.csv")
ues_prolif['study']='v3_prolific'
ues_utl = pd.read_csv("data/v3_utl/questionnaires/ues_v3_utl/ues_all.csv")
ues_utl['study'] = 'v3_utl'
ues = pd.concat([ues_prolif, ues_utl])
ues = ues[['id_participant', 'session_id', 'condition', 'study', 'engagement_score']]
ues = ues.pivot(index=['id_participant', 'condition', 'study'], columns='session_id', values='engagement_score').reset_index()
ues = ues.rename(columns={i:f"engagement_score_s{i}" for i in [0, 2, 4, 5, 7, 9]})
ues = ues.rename(columns={"id_participant": "participant_id"})
ues['engagement'] = ues[[f"engagement_score_s{i}" for i in [0, 2, 4, 5, 7, 9]]].mean(axis=1)
ues

session_id,participant_id,condition,study,engagement_score_s0,engagement_score_s2,engagement_score_s4,engagement_score_s5,engagement_score_s7,engagement_score_s9,engagement
0,513,baseline,v3_utl,2.000000,2.000000,1.916667,2.250000,2.000000,2.000000,2.027778
1,516,baseline,v3_utl,2.500000,1.916667,1.333333,1.666667,1.416667,2.083333,1.819444
2,517,baseline,v3_utl,2.833333,1.750000,2.083333,1.833333,2.083333,2.500000,2.180556
3,518,zpdes,v3_utl,2.666667,2.666667,3.000000,2.666667,2.583333,2.333333,2.652778
4,520,baseline,v3_utl,2.000000,2.416667,2.416667,2.666667,2.666667,1.916667,2.347222
...,...,...,...,...,...,...,...,...,...,...
118,672,baseline,v3_prolific,1.750000,1.583333,1.666667,1.666667,2.666667,2.916667,2.041667
119,674,zpdes,v3_prolific,2.333333,1.916667,1.583333,1.250000,1.750000,1.500000,1.722222
120,676,baseline,v3_prolific,3.000000,3.083333,3.000000,2.916667,3.083333,3.000000,3.013889
121,677,baseline,v3_prolific,4.083333,3.583333,1.833333,2.166667,2.166667,3.916667,2.958333


In [54]:
sims_prolif = pd.read_csv("data/v3_prolific/questionnaires/sims/sims.csv")
sims_prolif['study']='v3_prolific'
sims_utl = pd.read_csv("data/v3_utl/questionnaires/sims_v3_utl/sims_all.csv")
sims_utl['study'] = 'v3_utl'
sims = pd.concat([sims_prolif, sims_utl])
sims['SDI'] = (2 *sims['Intrinsic_motivation'] + sims['Identified_regulation']) - (
            2 * sims['Amotivation'] + sims['External_regulation'])
sims = sims[['id_participant', 'session_id', 'condition', 'study', 'SDI']]
sims = sims.pivot(index=['id_participant', 'condition', 'study'], columns='session_id', values='SDI').reset_index()
sims = sims.rename(columns={i:f"SDI_s{i}" for i in [1, 4, 5, 8]})
sims = sims.rename(columns={"id_participant": "participant_id"})
sims['SDI'] = sims[[f"SDI_s{i}" for i in [1, 4, 5, 8]]].mean(axis=1)
sims

session_id,participant_id,condition,study,SDI_s1,SDI_s4,SDI_s5,SDI_s8,SDI
0,513,baseline,v3_utl,5.125,1.375,3.500,5.625,3.90625
1,516,baseline,v3_utl,-13.250,-11.625,-11.375,-11.500,-11.93750
2,517,baseline,v3_utl,1.000,-4.500,-8.125,-9.500,-5.28125
3,518,zpdes,v3_utl,7.625,10.250,5.875,6.875,7.65625
4,520,baseline,v3_utl,0.625,2.750,8.750,2.500,3.65625
...,...,...,...,...,...,...,...,...
117,672,baseline,v3_prolific,8.250,-11.375,-5.500,0.875,-1.93750
118,674,zpdes,v3_prolific,-1.500,-2.500,-2.250,-0.750,-1.75000
119,676,baseline,v3_prolific,6.125,8.750,10.875,9.375,8.78125
120,677,baseline,v3_prolific,11.000,-3.500,-0.500,12.500,4.87500


In [55]:
df = pd.merge(ues, sims)
df = pd.merge(df, nasa)
df = pd.merge(df, cog_battery)
df = pd.merge(df, intra)

In [56]:
df.to_csv('sem_all_data.csv')

In [57]:
df[df['condition']=="baseline"].to_csv("v3_prolific_v3_utl_baseline_sem.csv")
df[df['condition']=="zpdes"].to_csv("v3_prolific_v3_utl_zpdes_sem.csv")

utl = df[df['study']=="v3_utl"]
features = pd.read_csv("data/v3_utl/feature_csv_v3_utl.csv")
features = features.rename(columns={"Unnamed: 0": "participant_id"})
utl = pd.merge(utl, features)
utl[utl['condition']=="baseline"].to_csv("v3_utl_baseline_sem.csv")
utl[utl['condition']=="zpdes"].to_csv("v3_utl_zpdes_sem.csv")


prolific = df[df['study']=="v3_prolific"]
prolific[prolific['condition']=="baseline"].to_csv("v3_prolific_baseline_sem.csv")
prolific[prolific['condition']=="zpdes"].to_csv("v3_prolific_zpdes_sem.csv")

,participant_id,idle,nb_episodes,age,study_duration,first_activity_n_targets,first_activity_speed,first_activity_tracking,first_activity_probe,first_activity_radius,last_activity_n_targets,last_activity_speed,last_activity_tracking,last_activity_probe,last_activity_radius,adhd,condition
0,518,18.985269,1436,67,3,2.490000,3.205000,3.895000,9.360000,0.987000,2.510000,3.930000,5.050000,8.920000,0.731000,0,zpdes
1,523,5.022889,1441,67,0,2.534483,3.310345,3.974138,10.793103,0.982759,2.350000,4.030000,5.600000,9.320000,0.669000,0,zpdes
2,524,8.430885,1038,70,0,2.460000,3.290000,4.020000,9.590000,0.971000,2.480000,4.245000,4.905000,9.180000,0.787000,0,zpdes
3,532,3.727270,1672,73,1,2.520000,2.345000,3.445000,11.210000,1.115000,3.090000,4.480000,4.845000,9.180000,0.735000,1,zpdes
4,533,1.916093,1837,67,0,2.640000,2.605000,4.235000,9.570000,0.920000,2.312500,4.203125,5.161458,8.739583,0.698958,0,zpdes
5,534,4.804080,1504,67,1,2.310000,3.680000,4.315000,9.330000,0.914000,2.423077,4.416667,5.448718,9.012821,0.676923,0,zpdes
6,535,10.323001,752,65,1,2.360000,3.380000,4.460000,9.640000,0.936000,2.950000,3.705000,4.720000,9.160000,0.836000,0,zpdes
7,539,14.956868,629,85,3,2.412371,2.298969,3.170103,11.443299,1.148454,2.735849,2.509434,3.783019,10.622642,1.094340,0,zpdes
8,541,6.225886,1430,66,0,2.820000,3.260000,3.480000,10.820000,1.022000,2.420000,3.995000,4.905000,9.400000,0.834000,1,zpdes
9,542,3.349197,1862,64,0,2.880000,3.395000,4.260000,9.750000,1.027000,3.310000,4.080000,4.560000,10.050000,0.806000,1,zpdes


In [32]:
def BF(BIC1, BIC2, name):
    print(name, np.exp(0.05*(BIC2-BIC1)))
BF(47.285, 51.549, name="zpdes, covar")
BF(47.285, 74.80, name="zpdes, var_i")
BF(47.285, 50.96, name="zpdes, slope_i")
print("  ")
BF(22.649,25.82, name="ctrl, covar")
BF(22.649, 38.51, name="ctrl, var_i")
BF(22.649, 22.65, name="ctrl, slope_i")

zpdes, covar 1.2376321529231926
zpdes, var_i 3.9580441431062394
zpdes, slope_i 1.201715356700392
  
ctrl, covar 1.1718105132861256
ctrl, var_i 2.210127044330951
ctrl, slope_i 1.0000500012500206


In [58]:
BIC_zpdes, BIC_control = 47.285, 22.649
# ZPDES: 
print("BFH0:")
BF(BIC1=BIC_zpdes, BIC2=41.504,name="pre_test")
BF(BIC1=BIC_zpdes, BIC2=44.256, name="age")
BF(BIC1=BIC_zpdes, BIC2=52.416, name="study_duration")
BF(BIC1=BIC_zpdes, BIC2=48.083, name="engagement")
BF(BIC1=BIC_zpdes, BIC2=51.185, name="motivation")
BF(BIC1=BIC_zpdes, BIC2=53.202, name="cognitive load")

print("BFH1:")
BF(BIC1=41.504,BIC2=BIC_zpdes, name="pre_test")
BF(BIC1=44.256,BIC2=BIC_zpdes,  name="age")
BF(BIC1=52.416,BIC2=BIC_zpdes, name="study_duration")
BF(BIC1=48.083,BIC2=BIC_zpdes, name="engagement")
BF(BIC1=51.185,BIC2=BIC_zpdes, name="motivation")
BF(BIC1=53.202,BIC2=BIC_zpdes, name="cognitive_load")

print("\n")

# Ctrl: 
print("BFH0:")
BF(BIC1=BIC_control, BIC2=27.179,name="pre_test")
BF(BIC1=BIC_control, BIC2=28.11, name="age")
BF(BIC1=BIC_control, BIC2=29.188, name="study_duration")
BF(BIC1=BIC_control, BIC2=28.277, name="engagement")
BF(BIC1=BIC_control, BIC2=27.944, name="motivation")
BF(BIC1=BIC_control, BIC2=29.566, name="cognitive load")
print("BFH1:")
BF(BIC1=27.179,BIC2=BIC_control, name="pre_test")
BF(BIC1=28.11, BIC2=BIC_control,  name="age")
BF(BIC1=29.188,BIC2=BIC_control, name="study_duration")
BF(BIC1=28.277,BIC2=BIC_control, name="engagement")
BF(BIC1=27.944,BIC2=BIC_control, name="motivation")
BF(BIC1=29.566,BIC2=BIC_control, name="cognitive_load")

BFH0:
pre_test 0.7489747557286489
age 0.859460854241331
study_duration 1.292463387353495
engagement 1.0407066983188495
motivation 1.2153109864897311
cognitive load 1.344268501433061
BFH1:
pre_test 1.3351584847838274
age 1.1635201243490334
study_duration 0.773716307776922
engagement 0.9608855229003457
motivation 0.8228346580560182
cognitive_load 0.7438990045024095


BFH0:
pre_test 1.2542026098339034
age 1.3139659414793832
study_duration 1.3867321388399159
engagement 1.3249834913472487
motivation 1.303105158763248
cognitive load 1.4131906213252887
BFH1:
pre_test 0.7973193423129872
age 0.761054733940903
study_duration 0.7211197981151282
engagement 0.7547263845402299
motivation 0.7673977754405337
cognitive_load 0.7076186219394812


In [5]:
import arviz as az
study="v3_utl"
task="moteval"
path = f"outputs/{study}/cognitive_battery/{task}"
task_condition = "total-task"
model = "hierar_binom_covar_pre"
trace = az.from_json(f"{path}/{task_condition}-{model}_inference_data.json")

In [7]:
import json
with open('analysis_scripts/cognitive_battery/hierarchical_bayesian_models/config/main_config_model_fit.JSON',
          'r') as f:
    all_conditions = json.load(f)

In [9]:
all_conditions['accuracy']

{'loadblindness': {'conditions': ['total-task'],
  '_models': ['hierar_binom_n_var'],
  'models': ['hierar_binom_covar_pre',
   'hierar_binom_n_gain',
   'hierar_binom_n_var']},
 'enumeration': {'conditions': ['total-task'],
  '_models': ['hierar_binom_n_var'],
  'models': ['hierar_binom_covar_pre',
   'hierar_binom_n_gain',
   'hierar_binom_n_var']},
 'memorability': {'conditions': ['total-task-hit'],
  '_models': ['hierar_binom_n_var'],
  'models': ['hierar_binom_covar_pre',
   'hierar_binom_n_gain',
   'hierar_binom_n_var']},
 'workingmemory': {'conditions': ['total-task'],
  '_models': ['hierar_binom_n_var'],
  'models': ['hierar_binom_covar_pre',
   'hierar_binom_n_gain',
   'hierar_binom_n_var']},
 'gonogo': {'conditions': ['GO'],
  '_models': ['hierar_binom_n_var'],
  'models': ['hierar_binom_covar_pre',
   'hierar_binom_n_gain',
   'hierar_binom_n_var']},
 'ufov': {'conditions': ['final'], 'models': ['precision_normal']},
 'moteval': {'conditions': ['total-task'],
  '_models': 